In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import geopandas as gpd

### 22.12.05 记录：

### 数据说明：

- 包含时间,地理信息的panel: `park_time_panel.csv`

- panel(包含`duration`): `park_panel.csv`, `park_panel.parquet`

- 不含`duration`的panel: `1204_panel.csv`,`1204_panel.parquet`

- 空panel(运行`duration2`函数用):`void_panel.csv` ：直接读取出来的不能用，需要先转换为tibble格式那种，代码如下：
 
void_panel = void_panel.groupby(["post_id","interval60"]).agg("first")

- 最新的原始数据(API抓下来的原始版本)：`2021_q2_new.csv`

- 最新的有效车位id： `1202_valid.csv`

- 最新的处理过后的数据集：`1204_res.csv`,`1204_res.parquet`

---------------------------------------

1. 读取数据

- 所有车位meters的geojson

In [3]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\Parking_Meters.geojson'
meters = gpd.read_file(path)

In [4]:
meters

,jurisdiction,collection_subroute_desc,smart_meter_flag,legislation_ref,pmr_route,parity_digit_position,latitude,ms_space_num,nfc_key,reason_code,...,meter_vendor,meter_model,sensor_flag,ms_pay_station_id,spt_code,old_rate_area,work_order,blockface_id,comments,geometry
0,SFMTA,None,N,None,G-4,2,37.7989496065,0.0,None,U4,...,-,-,N,-,None,-,None,363041.0,None,POINT (-122.40839 37.79895)
1,SFMTA,None,Y,None,F-2,0,37.8001665984,50.0,5L4X,UE,...,MacKay,Elite,N,916-00002,None,-,None,607322.0,None,POINT (-122.43913 37.80017)
2,SFMTA,None,Y,None,R-2,2,37.7666686018,0.0,2XKW,UC,...,IPS,M5,N,-,None,-,None,700032.0,None,POINT (-122.42217 37.76667)
3,SFMTA,None,N,None,E-2,2,37.787612803,0.0,GK56,U4,...,-,-,N,-,None,-,None,614072.0,None,POINT (-122.41428 37.78761)
4,SFMTA,None,N,None,N-2,2,37.776646302,0.0,3TT4,U1,...,-,-,N,-,None,-,None,208012.0,None,POINT (-122.41230 37.77665)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35610,SFMTA,None,N,None,J-5,2,37.7730383006,0.0,3ANP,U9,...,-,-,N,-,None,-,None,590111.0,-,POINT (-122.43767 37.77304)
35611,SFMTA,"9th St: Unit, 200",Y,None,N-2,2,37.7769272019,0.0,None,None,...,MacKay,Tango,N,209-00005,None,None,None,209001.0,None,POINT (-122.41544 37.77693)
35612,SFMTA,None,N,None,I-2,2,37.7906588034,0.0,None,U5,...,-,-,N,-,None,-,None,651001.0,None,POINT (-122.40072 37.79066)
35613,SFMTA,Townsend/Alameda,Y,None,S-1,2,37.7680695978,0.0,392S,-,...,IPS,M5,N,-,None,Area 3,None,692012.0,None,POINT (-122.40685 37.76807)


- 有效id

In [6]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1202_valid.csv'
valid_id = pd.read_csv(path)

In [7]:
valid_id

,Unnamed: 0,post_id
0,0,211-02380
1,1,596-00410
2,2,133-17960
3,3,650-33160
4,4,831-04430
...,...,...
15889,15889,722-29080
15890,15890,470-01210
15891,15891,470-01230
15892,15892,470-01250


- 读取park_panel

In [15]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\park_panel.csv'
park_panel=pd.read_csv(path)

In [18]:
park_panel

,post_id,interval60,duration
0,102-02990,2021-04-01 09:00:00,1208
1,102-02990,2021-04-01 10:00:00,3600
2,102-02990,2021-04-01 11:00:00,2392
3,102-02990,2021-04-01 12:00:00,2250
4,102-02990,2021-04-01 13:00:00,1800
...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0
13017182,920-00350,2021-06-30 14:00:00,887
13017183,920-00350,2021-06-30 15:00:00,2770
13017184,920-00350,2021-06-30 16:00:00,0


- 在park_panel中添加一些时间相关的列

In [19]:
p1 = park_panel

In [20]:
p1

,post_id,interval60,duration
0,102-02990,2021-04-01 09:00:00,1208
1,102-02990,2021-04-01 10:00:00,3600
2,102-02990,2021-04-01 11:00:00,2392
3,102-02990,2021-04-01 12:00:00,2250
4,102-02990,2021-04-01 13:00:00,1800
...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0
13017182,920-00350,2021-06-30 14:00:00,887
13017183,920-00350,2021-06-30 15:00:00,2770
13017184,920-00350,2021-06-30 16:00:00,0


In [22]:
p1['interval60']=pd.to_datetime(p1['interval60'])
p1['date'] = p1['interval60'].dt.date
p1['Month']= p1['interval60'].dt.month
p1['day_of_week']=p1['interval60'].dt.strftime('%A')

In [24]:
p1['hour']=p1['interval60'].dt.time

In [25]:
p1

,post_id,interval60,duration,date,Month,day_of_week,hour
0,102-02990,2021-04-01 09:00:00,1208,2021-04-01,4,Thursday,09:00:00
1,102-02990,2021-04-01 10:00:00,3600,2021-04-01,4,Thursday,10:00:00
2,102-02990,2021-04-01 11:00:00,2392,2021-04-01,4,Thursday,11:00:00
3,102-02990,2021-04-01 12:00:00,2250,2021-04-01,4,Thursday,12:00:00
4,102-02990,2021-04-01 13:00:00,1800,2021-04-01,4,Thursday,13:00:00
...,...,...,...,...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0,2021-06-30,6,Wednesday,13:00:00
13017182,920-00350,2021-06-30 14:00:00,887,2021-06-30,6,Wednesday,14:00:00
13017183,920-00350,2021-06-30 15:00:00,2770,2021-06-30,6,Wednesday,15:00:00
13017184,920-00350,2021-06-30 16:00:00,0,2021-06-30,6,Wednesday,16:00:00


-------------------------------

2.选取有效id的数据

In [8]:
meters['valid'] = meters['post_id'].isin(valid_id['post_id'])

In [10]:
meters = meters.loc[meters['valid']==True]

In [11]:
meters

,jurisdiction,collection_subroute_desc,smart_meter_flag,legislation_ref,pmr_route,parity_digit_position,latitude,ms_space_num,nfc_key,reason_code,...,meter_model,sensor_flag,ms_pay_station_id,spt_code,old_rate_area,work_order,blockface_id,comments,geometry,valid
53,SFMTA,None,Y,None,N-1,2,37.7722512008,0.0,7MUQ,-,...,M5,N,-,None,Area 3,None,211022.0,None,POINT (-122.41466 37.77225),True
72,SFMTA,None,Y,None,N-1,2,37.7721898005,0.0,5D7B,-,...,M5,N,-,None,Area 3,None,596001.0,None,POINT (-122.41966 37.77219),True
104,SFMTA,Noriega 30th Ave - 34th Ave,Y,None,A-2,2,37.7538484976,0.0,8C64,-,...,M5,N,-,None,Area 3,None,133172.0,None,POINT (-122.49143 37.75385),True
105,SFMTA,Divisadero - Sacramento,Y,None,C-2,2,37.788149,0.0,4EKO,-,...,M5,N,-,None,Area 3,None,650332.0,None,POINT (-122.44771 37.78815),True
108,PORT,South Embarcadero 400-700,Y,None,Port 2,2,37.7884178021,0.0,6AX9,-,...,M5,Y,-,None,Port 7,None,831042.0,None,POINT (-122.38773 37.78842),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35604,SFMTA,Townsend/Alameda,Y,None,S-1,2,37.7677518007,0.0,3HKC,-,...,M5,N,-,None,Area 3,None,692012.0,None,POINT (-122.40682 37.76775),True
35605,SFMTA,Polk 1800-2400,Y,None,D-2,2,37.7994473026,0.0,3B26,-,...,M5,N,-,None,Area 3,None,612241.0,None,POINT (-122.42255 37.79945),True
35607,SFMTA,Noriega 30th Ave - 34th Ave,Y,None,A-2,2,37.7537343976,0.0,ATJD,-,...,M5,N,-,None,Area 3,None,581242.0,None,POINT (-122.49018 37.75373),True
35613,SFMTA,Townsend/Alameda,Y,None,S-1,2,37.7680695978,0.0,392S,-,...,M5,N,-,None,Area 3,None,692012.0,None,POINT (-122.40685 37.76807),True


- 选取需要的列

In [12]:
valid_meters = meters.loc[:,['post_id','street_seg_ctrln_id','street_id','blockface_id','longitude','latitude','geometry']]

In [13]:
valid_meters

,post_id,street_seg_ctrln_id,street_id,blockface_id,longitude,latitude,geometry
53,211-02380,513000.0,211.0,211022.0,-122.4146590007,37.7722512008,POINT (-122.41466 37.77225)
72,596-00410,10047000.0,596.0,596001.0,-122.4196619016,37.7721898005,POINT (-122.41966 37.77219)
104,133-17960,1731000.0,133.0,133172.0,-122.4914245994,37.7538484976,POINT (-122.49143 37.75385)
105,650-33160,11265000.0,650.0,650332.0,-122.4477048998,37.788149,POINT (-122.44771 37.78815)
108,831-04430,12549201.0,831.0,831042.0,-122.3877260012,37.7884178021,POINT (-122.38773 37.78842)
...,...,...,...,...,...,...,...
35604,692-01500,13036000.0,692.0,692012.0,-122.4068219994,37.7677518007,POINT (-122.40682 37.76775)
35605,612-24310,10580000.0,612.0,612241.0,-122.4225540997,37.7994473026,POINT (-122.42255 37.79945)
35607,581-24440,9665000.0,581.0,581242.0,-122.4901839995,37.7537343976,POINT (-122.49018 37.75373)
35613,692-01220,13036000.0,692.0,692012.0,-122.4068522972,37.7680695978,POINT (-122.40685 37.76807)


- 统计每个street segment的有效车位数量

In [48]:
count_valid = valid_meters.groupby(['street_seg_ctrln_id'],as_index=False)['post_id'].count()

In [53]:
count_valid = count_valid.rename(columns={'post_id':'count_valid'})

In [54]:
count_valid

,street_seg_ctrln_id,count_valid
0,100000.0,4
1,1000000.0,4
2,1001000.0,2
3,10046000.0,10
4,10047000.0,10
...,...,...
1704,9900000.0,4
1705,9918000.0,6
1706,9919000.0,12
1707,9920000.0,9


- 建立一个时间panel

In [26]:
park_time_panel = pd.merge(p1,valid_meters,on='post_id')

In [31]:
park_time_panel

,post_id,interval60,duration,date,Month,day_of_week,hour,street_seg_ctrln_id,street_id,blockface_id,longitude,latitude,geometry
0,102-02990,2021-04-01 09:00:00,1208,2021-04-01,4,Thursday,09:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328)
1,102-02990,2021-04-01 10:00:00,3600,2021-04-01,4,Thursday,10:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328)
2,102-02990,2021-04-01 11:00:00,2392,2021-04-01,4,Thursday,11:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328)
3,102-02990,2021-04-01 12:00:00,2250,2021-04-01,4,Thursday,12:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328)
4,102-02990,2021-04-01 13:00:00,1800,2021-04-01,4,Thursday,13:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0,2021-06-30,6,Wednesday,13:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776)
13017182,920-00350,2021-06-30 14:00:00,887,2021-06-30,6,Wednesday,14:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776)
13017183,920-00350,2021-06-30 15:00:00,2770,2021-06-30,6,Wednesday,15:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776)
13017184,920-00350,2021-06-30 16:00:00,0,2021-06-30,6,Wednesday,16:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776)


In [55]:
park_time_panel = pd.merge(park_time_panel,count_valid,on="street_seg_ctrln_id")

In [56]:
park_time_panel

,post_id,interval60,duration,date,Month,day_of_week,hour,street_seg_ctrln_id,street_id,blockface_id,longitude,latitude,geometry,count_valid
0,102-02990,2021-04-01 09:00:00,1208,2021-04-01,4,Thursday,09:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328),1
1,102-02990,2021-04-01 10:00:00,3600,2021-04-01,4,Thursday,10:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328),1
2,102-02990,2021-04-01 11:00:00,2392,2021-04-01,4,Thursday,11:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328),1
3,102-02990,2021-04-01 12:00:00,2250,2021-04-01,4,Thursday,12:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328),1
4,102-02990,2021-04-01 13:00:00,1800,2021-04-01,4,Thursday,13:00:00,121000.0,102.0,102021.0,-122.4602134989,37.7832752977,POINT (-122.46021 37.78328),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0,2021-06-30,6,Wednesday,13:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776),32
13017182,920-00350,2021-06-30 14:00:00,887,2021-06-30,6,Wednesday,14:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776),32
13017183,920-00350,2021-06-30 15:00:00,2770,2021-06-30,6,Wednesday,15:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776),32
13017184,920-00350,2021-06-30 16:00:00,0,2021-06-30,6,Wednesday,16:00:00,10667101.0,615.0,615091.0,-122.4060795975,37.757763802,POINT (-122.40608 37.75776),32


In [57]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\park_time_panel.csv'
park_time_panel.to_csv(path)

In [47]:
park_time_panel.groupby(['post_id','date'],as_index=False)['duration'].sum()

,post_id,date,duration
0,102-02990,2021-04-01,22412
1,102-02990,2021-04-02,28015
2,102-02990,2021-04-03,18877
3,102-02990,2021-04-04,0
4,102-02990,2021-04-05,1320
...,...,...,...
1446349,920-00350,2021-06-26,0
1446350,920-00350,2021-06-27,0
1446351,920-00350,2021-06-28,0
1446352,920-00350,2021-06-29,12463
